In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Reshape, GlobalAveragePooling2D, Activation,UpSampling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

from mediapipe.python.solutions import pose as mp_pose
import cv2
pose_tracker = mp_pose.Pose()

In [2]:
train_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN/'
valid_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID/'
test_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST/'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=32)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

Found 17640 images belonging to 15 classes.
Found 5040 images belonging to 15 classes.
Found 2520 images belonging to 15 classes.


In [9]:
import os

main_folder = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN/"

# Iterate over subfolders in the main folder
for root, dirs, files in os.walk(main_folder):
    # Exclude the main folder itself from the count
    if root != main_folder:
        subfolder_name = os.path.basename(root)
        image_count = len(files)
        print(f"Subfolder: {subfolder_name}, Image Count: {image_count}")


Subfolder: calling, Image Count: 1176
Subfolder: clapping, Image Count: 1176
Subfolder: cycling, Image Count: 1176
Subfolder: dancing, Image Count: 1176
Subfolder: drinking, Image Count: 1176
Subfolder: eating, Image Count: 1176
Subfolder: fighting, Image Count: 1176
Subfolder: hugging, Image Count: 1176
Subfolder: laughing, Image Count: 1176
Subfolder: listening_to_music, Image Count: 1176
Subfolder: running, Image Count: 1176
Subfolder: sitting, Image Count: 1176
Subfolder: sleeping, Image Count: 1176
Subfolder: texting, Image Count: 1176
Subfolder: using_laptop, Image Count: 1176


In [10]:
import os

main_folder = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID/"

# Iterate over subfolders in the main folder
for root, dirs, files in os.walk(main_folder):
    # Exclude the main folder itself from the count
    if root != main_folder:
        subfolder_name = os.path.basename(root)
        image_count = len(files)
        print(f"Subfolder: {subfolder_name}, Image Count: {image_count}")

Subfolder: calling, Image Count: 336
Subfolder: clapping, Image Count: 336
Subfolder: cycling, Image Count: 336
Subfolder: dancing, Image Count: 336
Subfolder: drinking, Image Count: 336
Subfolder: eating, Image Count: 336
Subfolder: fighting, Image Count: 336
Subfolder: hugging, Image Count: 336
Subfolder: laughing, Image Count: 336
Subfolder: listening_to_music, Image Count: 336
Subfolder: running, Image Count: 336
Subfolder: sitting, Image Count: 336
Subfolder: sleeping, Image Count: 336
Subfolder: texting, Image Count: 336
Subfolder: using_laptop, Image Count: 336


In [11]:
import os

main_folder = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST/"

# Iterate over subfolders in the main folder
for root, dirs, files in os.walk(main_folder):
    # Exclude the main folder itself from the count
    if root != main_folder:
        subfolder_name = os.path.basename(root)
        image_count = len(files)
        print(f"Subfolder: {subfolder_name}, Image Count: {image_count}")

Subfolder: calling, Image Count: 168
Subfolder: clapping, Image Count: 168
Subfolder: cycling, Image Count: 168
Subfolder: dancing, Image Count: 168
Subfolder: drinking, Image Count: 168
Subfolder: eating, Image Count: 168
Subfolder: fighting, Image Count: 168
Subfolder: hugging, Image Count: 168
Subfolder: laughing, Image Count: 168
Subfolder: listening_to_music, Image Count: 168
Subfolder: running, Image Count: 168
Subfolder: sitting, Image Count: 168
Subfolder: sleeping, Image Count: 168
Subfolder: texting, Image Count: 168
Subfolder: using_laptop, Image Count: 168


In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.mobilenet import preprocess_input
from mediapipe.python.solutions import pose as mp_pose
import tensorflow as tf

In [4]:
import os
import shutil
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from mediapipe.python.solutions import pose as mp_pose

train_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN/"
batch_size = 32

# Initialize the pose tracker
pose_tracker = mp_pose.Pose()

# Create a dictionary to map subclass folders to class labels
subclass_to_class = {
    "sitting": 0,
    "sleeping": 1,
    "texting": 2,
    "using_laptop": 3,
    "running": 4,
    "listening_to_music": 5,
    "laughing": 6,
    "fighting": 7,
    "hugging": 8,
    "eating": 9,
    "drinking": 10,
    "dancing": 11,
    "clapping": 12,
    "cycling": 13,
    "calling": 14
}

def data_generator(train_dir, batch_size):
    # Initialize the pose tracker
    pose_tracker = mp_pose.Pose()

    while True:
        images = []
        landmarks = []
        class_labels = []

        # Iterate over the source directory and process images and landmarks
        for root, dirs, files in os.walk(train_dir):
            for file in files:
                if file.endswith(".jpg") or file.endswith(".png"):
                    train_file = os.path.join(root, file)

                    # Process the image to extract landmarks
                    image = cv2.imread(train_file)
                    results = pose_tracker.process(image)
                    pose_landmarks = results.pose_landmarks

                    # Extract landmarks and perform further processing if landmarks are found
                    if pose_landmarks is not None and len(pose_landmarks.landmark) == 33:
                        # Apply preprocessing to the image
                        preprocessed_image = tf.keras.applications.mobilenet.preprocess_input(image)

                        # Store the image, landmarks, and class label
                        images.append(preprocessed_image)
                        landmark_coordinates = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
                        landmarks.append(landmark_coordinates)

                        subclass_folder = os.path.basename(os.path.dirname(train_file))
                        class_label = subclass_to_class.get(subclass_folder)
                        class_labels.append(class_label)

                if len(images) == batch_size:
                    # Convert target labels to one-hot encoded format
                    class_labels = to_categorical(class_labels, num_classes=15)

                    # Convert landmarks to a NumPy array
                    landmarks = np.array(landmarks)

                    # Append zeros and reshape landmarks array
                    landmarks = np.concatenate((landmarks, np.zeros((batch_size, 1024 - 33, 3))), axis=1)
                    landmarks = landmarks.reshape((batch_size, 32, 32, 3))

                    yield (np.array(images), landmarks, np.array(class_labels))
                    images = []
                    landmarks = []
                    class_labels = []

        if len(images) > 0:
            # Convert target labels to one-hot encoded format
            class_labels = to_categorical(class_labels, num_classes=15)

            # Convert landmarks to a NumPy array
            landmarks = np.array(landmarks)

            # Append zeros and reshape landmarks array
            landmarks = np.concatenate((landmarks, np.zeros((len(images), 1024 - 33, 3))), axis=1)
            landmarks = landmarks.reshape((len(images), 32, 32, 3))

            yield (np.array(images), landmarks, np.array(class_labels))


In [5]:
valid_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID/"
batch_size = 32
    

valid_generator = data_generator(valid_dir, batch_size)

In [6]:
test_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST/"
batch_size = 32
    

test_generator = data_generator(test_dir, batch_size)

In [7]:
train_generator = data_generator(train_dir, batch_size)

# Retrieve and print the first batch
batch_images, batch_landmarks, batch_labels = next(train_generator)
print("Batch Images:", batch_images.shape)
print("Batch Landmarks:", batch_landmarks.shape)
print("Batch Labels:", batch_labels.shape)



Batch Images: (32, 224, 224, 3)
Batch Landmarks: (32, 32, 32, 3)
Batch Labels: (32, 15)


In [8]:
train_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN/"
batch_size = 32

train_generator = data_generator(train_dir, batch_size)

# Calculate the total number of training images
total_images = len(os.listdir(train_dir))

# Calculate the number of steps required to iterate over all the training images
steps_per_epoch = total_images // batch_size + 1

print("Total Images:", total_images)
print("Steps per Epoch:", steps_per_epoch)

# Iterate over the training generator for the specified number of steps
for step in range(steps_per_epoch):
    batch_images, batch_landmarks, batch_labels = next(train_generator)
    print("Step:", step)
    print("Batch Images:", batch_images.shape)
    print("Batch Landmarks:", batch_landmarks.shape)
    print("Batch Labels:", batch_labels.shape)


Total Images: 15
Steps per Epoch: 1
Step: 0
Batch Images: (32, 224, 224, 3)
Batch Landmarks: (32, 32, 32, 3)
Batch Labels: (32, 15)


**MODEL BUILDING**

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, concatenate
from tensorflow.keras.models import Model

model1 = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
        trainable=True),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu'),
])

# Build the model
model1.build((None, 224, 224, 3))
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              4226432   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
Total params: 4,390,400
Trainable params: 4,366,000
Non-trainable params: 24,400
_________________________________________________________________


In [13]:
mobile = tf.keras.applications.MobileNetV2(
    input_shape=(32, 32, 3), weights=None, include_top=False)

In [14]:
x = mobile.layers[-1].output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling to reduce dimensions to (None, 1)
x = Dropout(0.1)(x)
output = Dense(128, activation='relu')(x)

model2 = Model(inputs=mobile.input, outputs=output)
model2.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 16, 16, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [15]:
combined = concatenate([model1.output,model2.output])
prediction = Dense(15, activation='softmax')(combined)

fin_model = Model(inputs=[model1.input,model2.input], outputs=prediction)
fin_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 16, 16, 32)   0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [16]:
fin_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
train_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN/"
valid_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID/"
batch_size = 32

train_generator = data_generator(train_dir, batch_size)
valid_generator = data_generator(valid_dir, batch_size)

num_epochs = 10
steps_per_epoch = sum([len(files) for _, _, files in os.walk(train_dir)]) // batch_size + 1

for epoch in range(num_epochs):
    for step in range(steps_per_epoch):
        images_batch, landmarks_batch, labels_batch = next(train_generator)
        valid_images_batch, valid_landmarks_batch, valid_labels_batch = next(valid_generator)
        fin_model.fit(
            [images_batch, landmarks_batch], 
            labels_batch, 
            validation_data=([valid_images_batch, valid_landmarks_batch], valid_labels_batch),
            verbose=1
        )
    
    # Perform actions after each epoch
    print("Epoch", epoch+1, "completed.")


1/1 [==============================] - 6s 6s/step - loss: 6.2300 - accuracy: 0.0000e+00 - val_loss: 4.3891 - val_accuracy: 0.0000e+00
